In [ ]:
import DifferentialEquations as DE

In [ ]:
datapath = "D:\\MCFdata\\w7x\\"
println( Threads.nthreads() )
using Jynamics
using TensorCast

In [ ]:
rminb = 4.0;
rmaxb = 7.0;
zminb = -1.5;
zmaxb = 1.5;
nSym = 5;

R = collect( range(rminb, stop=rmaxb, length=128) );
Z = collect( range(zminb, stop=zmaxb, length=128) );
Phi = collect( range(0.0, stop=2pi/nSym, length=65) );


In [ ]:
BR = Array{Float64,3}(undef, 128, 128, 65);
BZ = Array{Float64,3}(undef, 128, 128, 65);
BPhi = Array{Float64,3}(undef, 128, 128, 65);
BR_pert = Array{Float64,3}(undef, 128, 128, 65);
BZ_pert = Array{Float64,3}(undef, 128, 128, 65);
BPhi_pert = Array{Float64,3}(undef, 128, 128, 65);

using NPZ
Bfield_np = npzread(datapath * "w7x-op21-standard.npy");
Bfield_pert_np = npzread(datapath * "w7x-op21-controlCoils1.npy");

In [ ]:
for iPhi = 1:length(Phi)-1
    BR[:,:,iPhi] = Bfield_np[:,:,iPhi,1];
    BZ[:,:,iPhi] = Bfield_np[:,:,iPhi,2];
    BPhi[:,:,iPhi] = Bfield_np[:,:,iPhi,3];
end
BR[:,:,end] = Bfield_np[:,:,1,1];
BZ[:,:,end] = Bfield_np[:,:,1,2];
BPhi[:,:,end] = Bfield_np[:,:,1,3];

B_unperturbed = CylindricalVectorField(R, Z, Phi, BR, BZ, BPhi; nSym=nSym);


In [ ]:
for iPhi = 1:length(Phi)-1
    BR_pert[:,:,iPhi] = Bfield_pert_np[:,:,iPhi,1];
    BZ_pert[:,:,iPhi] = Bfield_pert_np[:,:,iPhi,2];
    BPhi_pert[:,:,iPhi] = Bfield_pert_np[:,:,iPhi,3];
end
BR_pert[:,:,end] = Bfield_pert_np[:,:,1,1];
BZ_pert[:,:,end] = Bfield_pert_np[:,:,1,2];
BPhi_pert[:,:,end] = Bfield_pert_np[:,:,1,3];

B_pert = CylindricalVectorField(R, Z, Phi, BR_pert, BZ_pert, BPhi_pert; nSym=5);
# B_pert = 0.2*B_pert

In [ ]:
B_total = B_unperturbed + B_pert

In [ ]:
import DifferentialEquations as DE

In [ ]:
# using GridInterpolations
using Interpolations

In [ ]:

BR_interp = linear_interpolation((R,Z,Phi), BR);
BZ_interp = linear_interpolation((R,Z,Phi), BZ);
BPhi_interp = linear_interpolation((R,Z,Phi), BPhi);

In [ ]:
using TensorCast
using Memoization

In [ ]:
@cast RBRoBPhi[iR,iZ,iPhi] := R[iR] * BR[iR,iZ,iPhi] / BPhi[iR,iZ,iPhi];
@cast RBZoBPhi[iR,iZ,iPhi] := R[iR] * BZ[iR,iZ,iPhi] / BPhi[iR,iZ,iPhi];

In [ ]:
size(RBRoBPhi)

In [ ]:
dR = R[2] - R[1]
dZ = Z[2] - Z[1]

        

In [ ]:

FLT_A =  RVpoloVPhi_pRpZ_interp(B_unperturbed)

In [ ]:
FLT_A(5.0, 0.0, 0.0)

In [ ]:
condition(x,t,integrator) = x[1]>R[end-3] || x[1] < R[3] || x[2] >Z[end-3] || x[2] < Z[3]
affect!(integrator) = DE.terminate!(integrator)
cb = DE.DiscreteCallback(condition,affect!)

In [ ]:
import DifferentialEquations as DE
import LinearAlgebra as LA



In [ ]:
# X-points of the edge island chain
# cyc_init_rzphi = lookfor_cycle(B_unperturbed, [6.05; 0.6; 0.0])
# cyc_init_rzphi = lookfor_cycle(0.0, [6.05;-0.6; 0.0])
# cyc_init_rzphi = lookfor_cycle(0.0, [5.5; 0.8; 0.0])
# cyc_init_rzphi = lookfor_cycle(0.0, [5.5;-0.8; 0.0])
# cyc_init_rzphi = lookfor_cycle(0.0, [5.7; 0.0; 0.0])

# O-point of the magnetic axis
cyc_init_rzphi = lookfor_cycle(B_unperturbed, [5.92; 0.0; 0.0])

In [ ]:
phi = cyc_init_rzphi[3]
tor_turn = 1
tspan = (phi, phi + tor_turn*2pi)

sols_dict = create_tracing_sols_Dict(B_unperturbed, cyc_init_rzphi[1:2], cyc_init_rzphi[3])

prob_Xpol = DE.ODEProblem(dXpol_dphi!_generator(sols_dict), cyc_init_rzphi[1:2], tspan)
sol_Xpol = DE.solve(prob_Xpol, DE.RK4(), 
    abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128, callback=cb) 
sols_dict["sol_Xpol"] = sol_Xpol

prob_DXpol = DE.ODEProblem(dDXpol_dphi!_generator(sols_dict), [1 0; 0 1], tspan)
# prob_DXpol = DE.ODEProblem(dDXpol_dphi!, [1 0; 0 1], tspan, [sol_Xpol,])
sol_DXpol = DE.solve(prob_DXpol, DE.RK4(), 
    abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128,)
sols_dict["sol_DXpol"] = sol_DXpol

In [ ]:

prob_DPm = DE.ODEProblem(dDPm_dphi!_generator(sols_dict), sol_DXpol(tspan[2]), tspan,)
sol_DPm = DE.solve(prob_DPm, DE.RK4(), 
    abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128,)
sols_dict["sol_DPm"] = sol_DPm

In [ ]:

sols_dict["delta_v"] = B_pert
prob_delta_Xpol = DE.ODEProblem(d_delta_Xpol_dphi!_generator(sols_dict), [0;0], tspan)
sol_delta_Xpol = DE.solve(prob_delta_Xpol, DE.RK4(), 
    abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128,)
sols_dict["sol_delta_Xpol"] = sol_delta_Xpol

In [ ]:
Xcyc_init_delta_Delta = LA.inv( sol_DPm(phi) - LA.I(2) ) * (- sol_delta_Xpol(2pi) )

In [ ]:

prob_delta_Xcyc = DE.ODEProblem(d_delta_Xcyc_dphi!_generator(sols_dict), Xcyc_init_delta_Delta, tspan)
sol_delta_Xcyc = DE.solve(prob_delta_Xcyc, DE.RK4(), 
    abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128,)
sols_dict["sol_delta_Xcyc"] = sol_delta_Xcyc

In [ ]:
phi = cyc_init_rzphi[3]
tor_turn = 800
tspan = (phi, phi + tor_turn*2pi)

flux_surface_dicts = [create_tracing_sols_Dict(B_unperturbed, 
    [cyc_init_rzphi[1]+dR, cyc_init_rzphi[2]], cyc_init_rzphi[3]) for dR in 0.09:0.06:0.26]
for dict in flux_surface_dicts
    dict["axrz"] = cyc_init_rzphi[1:2]
end
trajN = length(flux_surface_dicts)

using Base.Threads: @threads
@threads for traj_i in 1:trajN
    local prob_Xpol = DE.ODEProblem(dXpol_dphi!_generator(flux_surface_dicts[traj_i]), flux_surface_dicts[traj_i]["r0z0"], tspan)
    flux_surface_dicts[traj_i]["sol_Xpol"] = DE.solve(prob_Xpol, DE.RK4(), 
        abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128, callback=cb) 

    local prob_DXpol = DE.ODEProblem(dDXpol_dphi!_generator(flux_surface_dicts[traj_i]), [1 0; 0 1], tspan)
    flux_surface_dicts[traj_i]["sol_DXpol"] = DE.solve(prob_DXpol, DE.RK4(), 
        abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128,)

    flux_surface_dicts[traj_i]["delta_v"] = B_pert
    local prob_delta_Xpol = DE.ODEProblem(d_delta_Xpol_dphi!_generator(flux_surface_dicts[traj_i]), [0;0], tspan)
    flux_surface_dicts[traj_i]["sol_delta_Xpol"] = DE.solve(prob_delta_Xpol, DE.RK4(), 
        abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128,)
end


In [ ]:
traj_i = 1
scatxy = reduce( hcat, [flux_surface_dicts[traj_i]["sol_Xpol"](t) for t in flux_surface_dicts[traj_i]["sol_Xpol"].t[1]:2pi:flux_surface_dicts[traj_i]["sol_Xpol"].t[end]] )
ans = find_Delta_theta(scatxy, cyc_init_rzphi[1:2])

In [ ]:
length(flux_surface_dicts)

In [ ]:
import Plots
Plots.plot(aspect_ratio=1.0, size=(600,1000))

for traj_i in 1:trajN
    scatxy = reduce( hcat, [flux_surface_dicts[traj_i]["sol_Xpol"](t) for t in flux_surface_dicts[traj_i]["sol_Xpol"].t[1]:2pi:flux_surface_dicts[traj_i]["sol_Xpol"].t[end]]  )
    scatx, scaty = scatxy[1,:], scatxy[2,:]
    Plots.scatter!(scatx, scaty, ms=0.2)
end
Plots.scatter!()

In [ ]:
traj_i = 1
Nfourier = 10
a0_R, an_R, bn_R, a0_Z, an_Z, bn_Z = find_delta_chi_fourier_series(flux_surface_dicts[traj_i], sols_dict["sol_delta_Xcyc"](0.0)[2], Nfourier)
scatxy = reduce( hcat, [flux_surface_dicts[traj_i]["sol_Xpol"](t) for t in flux_surface_dicts[traj_i]["sol_Xpol"].t[1]:2pi:flux_surface_dicts[traj_i]["sol_Xpol"].t[end]] )
Δθ = find_Delta_theta(scatxy, cyc_init_rzphi[1:2])
print("Δθ = ", Δθ, "\n")

perturbed_flux_surface_dict = create_tracing_sols_Dict(B_total, 
    flux_surface_dicts[traj_i]["r0z0"] 
    + [fourier_series(0.0, a0_R, an_R, bn_R), fourier_series(0.0, a0_Z, an_Z, bn_Z)], 
    0.0)
perturbed_flux_surface_dict["axrz"] = cyc_init_rzphi[1:2]+sols_dict["sol_delta_Xcyc"](0.0)
prob_Xpol = DE.ODEProblem(dXpol_dphi!_generator(perturbed_flux_surface_dict), perturbed_flux_surface_dict["r0z0"], tspan)
perturbed_flux_surface_dict["sol_Xpol"] = DE.solve(prob_Xpol, DE.RK4(), 
    abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128, callback=cb) 
scatxy = reduce( hcat, [perturbed_flux_surface_dict["sol_Xpol"](t) for t in perturbed_flux_surface_dict["sol_Xpol"].t[1]:2pi:perturbed_flux_surface_dict["sol_Xpol"].t[end]] )
Δθ_perturbed = find_Delta_theta(scatxy, cyc_init_rzphi[1:2]+sols_dict["sol_delta_Xcyc"](0.0))
print("Δθ_perturbed = ", Δθ_perturbed, "\n")
print(sols_dict["sol_delta_Xcyc"](0.0)[2], "\n",  )
print(fourier_series(0.0, a0_Z, an_Z, bn_Z))
# Plots.plot(0:0.01:2pi, [fourier_series(θ, a0_R, an_R, bn_R) for θ in 0:0.01:2pi], linestyle=:dash)
# Plots.plot(0:0.01:2pi, [fourier_series(θ, a0_Z, an_Z, bn_Z) for θ in 0:0.01:2pi], linestyle=:dash)

# Nfourier = 10
# a0_R, an_R, bn_R, a0_Z, an_Z, bn_Z = find_chi_fourier_series(flux_surface_dicts[traj_i], Nfourier)
# Plots.plot!(0:0.01:2pi, [fourier_series(θ, a0_R, an_R, bn_R) for θ in 0:0.01:2pi])
# Plots.plot!(0:0.01:2pi, [fourier_series(θ, a0_Z, an_Z, bn_Z) for θ in 0:0.01:2pi])
# a0_R, an_R, bn_R, a0_Z, an_Z, bn_Z = find_chi_fourier_series(perturbed_flux_surface_dict, Nfourier)
# # Plots.plot!(0:0.01:2pi, [fourier_series(θ, a0_R, an_R, bn_R) for θ in 0:0.01:2pi])
# # Plots.plot!(0:0.01:2pi, [fourier_series(θ, a0_Z, an_Z, bn_Z) for θ in 0:0.01:2pi])
# a0_R, an_R, bn_R, a0_Z, an_Z, bn_Z = find_chi_fourier_series(perturbed_flux_surface_dict, Nfourier) .- find_chi_fourier_series(flux_surface_dicts[traj_i], Nfourier)
# print("delta_chi_z:", fourier_series(0.0, a0_Z, an_Z, bn_Z))
# Plots.plot!(0:0.01:2pi, [fourier_series(θ, a0_R, an_R, bn_R) for θ in 0:0.01:2pi])
# Plots.plot!(0:0.01:2pi, [fourier_series(θ, a0_Z, an_Z, bn_Z) for θ in 0:0.01:2pi])


In [ ]:
evolve_deltaX_dict = create_tracing_sols_Dict(B_unperturbed, flux_surface_dicts[traj_i]["r0z0"], 0.0)
evolve_deltaX_dict["delta_v"] = B_pert
Nfourier = 10
a0_R_fs, an_R_fs, bn_R_fs, a0_Z_fs, an_Z_fs, bn_Z_fs = find_chi_fourier_series(flux_surface_dicts[traj_i], Nfourier)

phi = 0.0
tspan = (phi, phi + 2pi/5)

_Theta = collect( range(0, stop=2pi, length=500) )
_Phi = collect( range(0, stop=2pi/5, length=5) )

deltaX_ThetaPhi = Array{Float64,3}(undef, length(_Theta), length(_Phi), 2)
X_ThetaPhi = Array{Float64,3}(undef, length(_Theta), length(_Phi), 2)

for iTheta in 1:length(_Theta)
    ϑ = _Theta[iTheta]

    x0rz = [fourier_series(ϑ, a0_R_fs, an_R_fs, bn_R_fs), fourier_series(ϑ, a0_Z_fs, an_Z_fs, bn_Z_fs)]
    deltaX_init = [fourier_series(ϑ, a0_R, an_R, bn_R), fourier_series(ϑ, a0_Z, an_Z, bn_Z)]
    evolve_deltaX_dict["r0z0"] = x0rz
    
    prob_Xpol = DE.ODEProblem(dXpol_dphi!_generator(evolve_deltaX_dict), x0rz, tspan)
    evolve_deltaX_dict["sol_Xpol"] = DE.solve(prob_Xpol, DE.RK4(), 
        abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128, callback=cb) 
    for (iPhi, phi) in enumerate(_Phi)
        X_ThetaPhi[iTheta, iPhi, :] .= evolve_deltaX_dict["sol_Xpol"](phi)
    end
    prob_delta_Xpol = DE.ODEProblem(d_delta_Xpol_dphi!_generator(evolve_deltaX_dict), deltaX_init, tspan)
    sol_delta_Xpol = DE.solve(prob_delta_Xpol, DE.RK4(), 
        abstol=1e-9, reltol=1e-6, maxiters=1e9, dt=pi/128,)
    for (iPhi, phi) in enumerate(_Phi)
        deltaX_ThetaPhi[iTheta, iPhi, :] .= sol_delta_Xpol(phi)
    end
end


In [ ]:
using NPZ

# 打包保存到 npz 文件
npzwrite("w7x_standard_configuration_flux_surface_deformation.npz", Dict(
    "Theta" => _Theta,
    "Phi" => _Phi,
    "deltaX_ThetaPhi" => deltaX_ThetaPhi,
    "X_ThetaPhi" => X_ThetaPhi
))


In [ ]:

VectorMatrix2Array(sol_DPm.u)[1,:,:]

In [ ]:
using HDF5
h5open(datapath * "w7x_standard_cycle1.h5", "w") do fid
    g = create_group(fid, "cyc")
    g["Xpol_t"] = sol_Xpol.t
    g["Xpol"] = VectorVector2Array(sol_Xpol.u)
    g["DXpol_t"] = sol_DXpol.t
    g["DXpol"] = VectorMatrix2Array(sol_DXpol.u)
    g["DPm_t"] = sol_DPm.t
    g["DPm"] = VectorMatrix2Array(sol_DPm.u)
    g["delta_Xcyc_t"] = sol_delta_Xcyc.t
    g["delta_Xcyc"] = VectorVector2Array(sol_delta_Xcyc.u)
    # g["delta_DXpol_init_Xcyc_t"] = sol_delta_DXpol_init_Xcyc.t
    # g["delta_DXpol_init_Xcyc"] = VectorMatrix2Array(sol_delta_DXpol_init_Xcyc.u)
    g["ddB_DPm_with_init_Xcyc_shift_t"] = sol_ddB_DPm_with_init_Xcyc_shift.t
    g["ddB_DPm_with_init_Xcyc_shift"] = VectorMatrix2Array(sol_ddB_DPm_with_init_Xcyc_shift.u)
end;